# โจทย์: Automate SQL ได้ไหม?


## SQLite with Python

In [1]:
# เปลี่ยนจาก sqlite3 เป็น sqlalchemy
from sqlalchemy import create_engine, text

# สร้าง engine แทนการใช้ connect โดยตรง
engine = create_engine('sqlite:///mysqlite01.db')

In [3]:
engine

Engine(sqlite:///mysqlite01.db)

In [4]:

# สร้างตาราง products ด้วย engine.execute
with engine.connect() as conn:
    conn.execute(text('''
    CREATE TABLE IF NOT EXISTS products (
        product_id INTEGER PRIMARY KEY,
        name TEXT NOT NULL,
        price REAL NOT NULL,
        category TEXT
    )
    '''))

    # สร้างตาราง orders
    conn.execute(text('''
    CREATE TABLE IF NOT EXISTS orders (
        order_id INTEGER PRIMARY KEY,
        product_id INTEGER,
        quantity INTEGER,
        order_date TEXT
    )
    '''))

In [5]:
with engine.connect() as conn:
    # เพิ่มข้อมูลในตาราง products
    conn.execute(text('''
        INSERT INTO products (name, price, category)
        VALUES ('iPhone 14', 35900, 'Electronics')
    '''))
    
    # เพิ่มข้อมูลในตาราง orders 
    conn.execute(text('''
        INSERT INTO orders (product_id, quantity, order_date)
        VALUES (1, 2, '2023-10-20')
    '''))
    
    conn.commit()

## Faker

In [6]:
from faker import Faker

# สร้าง Faker object สำหรับภาษาไทย
fake = Faker()

fake_data = {
    "name": fake.random_element(elements=("iPhone", "iPad", "MacBook", "AirPods")),
    "price": fake.random_int(min=100, max=50000),
    "category": fake.random_element(
        elements=("Electronics", "Clothing", "Food", "Books")
    ),
}
fake_data

{'name': 'AirPods', 'price': 647, 'category': 'Books'}

## Faker + SQLite

In [12]:
# สร้างข้อมูลปลอมด้วย Faker
fake_product = {
    "name": fake.random_element(elements=("iPhone", "iPad", "MacBook", "AirPods")),
    "price": fake.random_int(min=100, max=50000), 
    "category": fake.random_element(elements=("Electronics", "Clothing", "Food", "Books"))
}
fake_product

{'name': 'iPhone', 'price': 46210, 'category': 'Electronics'}

In [13]:
# เพิ่มข้อมูลลงใน SQLite
with engine.connect() as conn:
    conn.execute(text('''
        INSERT INTO products (name, price, category)
        VALUES (:name, :price, :category)
    '''), fake_product)
    conn.commit()

## Pandas

In [ ]:
import pandas as pd

table (dataframe) = index + column + data

|index|column1|column2|column3|
|---|---|---|---|
|0|data01|data02|data03|
|1|data11|data12|data13|

In [ ]:
# สร้าง DataFrame จากข้อมูลพื้นฐาน
data = {
    'name': ['สมชาย', 'สมหญิง', 'สมศรี'],
    'age': [25, 30, 35],
    'city': ['กรุงเทพ', 'เชียงใหม่', 'ภูเก็ต']
}
df = pd.DataFrame(data)
df

In [ ]:
name_series = df["name"]
name_series

In [ ]:
age_series = df["age"]
age_series

In [ ]:
df.info()

In [ ]:
# แสดงข้อมูลเบื้องต้นของ DataFrame
df.describe()

In [ ]:
# Python สามารถนำไปวิเคราะห์ข้อมูลต่อได้

## Pandas + SQLite

In [ ]:
data = {
    'name': ['สมชาย', 'สมหญิง', 'สมศรี'],
    'age': [25, 30, 35],
    'city': ['กรุงเทพ', 'เชียงใหม่', 'ภูเก็ต']
}
df = pd.DataFrame(data)
df

In [ ]:
df.to_sql('people', engine, if_exists='append', index=False)

## Pandas + SQLite + Faker

In [ ]:
# สร้างข้อมูลปลอมด้วย Faker
products = []   
for _ in range(100):
    product = {
        "name": fake.random_element(elements=("iPhone", "iPad", "MacBook", "AirPods")),
    "price": fake.random_int(min=100, max=50000), 
        "category": fake.random_element(elements=("Electronics", "Clothing", "Food", "Books"))
    }
    products.append(product)

# สร้าง DataFrame สำหรับสินค้า
df_products = pd.DataFrame(products)
df_products

In [ ]:
df_products.to_sql('products', engine, if_exists='append', index=False)

## Challege: สร้างข้อมูลลูกค้าโดยใช้ Faker และ Pandas และเพิ่มลงใน SQLite ทุก 5 วินาที

In [1]:
# faker to df
import faker
import pandas as pd
import time
import pendulum

# สร้าง Faker object สำหรับสร้างข้อมูลปลอม
fake = faker.Faker()
def generate_customers(num_customers):
    customers = []
    for _ in range(num_customers):  # สร้าง 5 รายการ
        customer = {
            'customer_id': fake.uuid4(),
            'name': fake.name(),
            'email': fake.email(), 
            'phone': fake.phone_number(),
            'address': fake.address(),
            'segment': fake.random_element(['Retail', 'Wholesale', 'Online', 'In-Store']),
            'join_date': pendulum.now('Asia/Bangkok'),
            'loyalty_points': fake.random_int(min=0, max=1000)
        }
        customers.append(customer)
    return customers
customers = generate_customers(2)
customer_df = pd.DataFrame(customers)
customer_df

,customer_id,name,email,phone,address,segment,join_date,loyalty_points
0,cd316474-ee4f-4679-882b-43fd896f3647,Jennifer Wilson DVM,suzannetodd@example.net,469-455-9054x228,Unit 0409 Box 7847\nDPO AE 68757,Retail,2024-11-29 19:57:53.199277+07:00,237
1,d8aeea22-1b0d-4562-a941-f37978efcd2e,John Watkins,william45@example.com,(636)640-7387x8283,"300 Griffin Plain Suite 503\nPort Hannah, KY 4...",Retail,2024-11-29 19:57:53.199836+07:00,935


In [ ]:
# df to sqlite

# เปลี่ยนจาก sqlite3 เป็น sqlalchemy
from sqlalchemy import create_engine, text

# สร้าง engine แทนการใช้ connect โดยตรง
engine = create_engine('sqlite:///mysqlite01.db')

# สร้างข้อมูลลูกค้า
while True:
    customers = generate_customers(5)
    customer_df = pd.DataFrame(customers)
    customer_df.to_sql('customers', engine, if_exists='append', index=False)
    print(f"Created {len(customers)} customers at {time.strftime('%H:%M:%S')}")
    time.sleep(5)

## Final App

In [ ]:
import faker
import pandas as pd
import time
# เปลี่ยนจาก sqlite3 เป็น sqlalchemy
from sqlalchemy import create_engine, text

# สร้าง engine แทนการใช้ connect โดยตรง
engine = create_engine('sqlite:///mysqlite01.db')
# สร้าง Faker object สำหรับสร้างข้อมูลปลอม
fake = faker.Faker()

def generate_customers(num_customers):
    customers = []
    for _ in range(num_customers):  # สร้าง 5 รายการ
        customer = {
            'customer_id': fake.uuid4(),
            'name': fake.name(),
            'email': fake.email(), 
            'phone': fake.phone_number(),
            'address': fake.address(),
            'segment': fake.random_element(['Retail', 'Wholesale', 'Online', 'In-Store']),
            'join_date': pendulum.now('Asia/Bangkok'),
            'loyalty_points': fake.random_int(min=0, max=1000)
        }
        customers.append(customer)
    return customers

# สร้างข้อมูลลูกค้า
while True:
    customers = generate_customers(5)
    customer_df = pd.DataFrame(customers)
    customer_df.to_sql('customers', engine, if_exists='append', index=False)
    print(f"Created {len(customers)} customers at {time.strftime('%H:%M:%S')}")
    time.sleep(5)

In [ ]:
# Python สามารถ generate data เข้า database แบบ realtime ได้แล้ว

## Congratulation!!